In [1]:
%load_ext autoreload
%autoreload 2

In [39]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import json
import os
load_dotenv()
import yfinance as yf
from langchain.tools import tool
import functools
from pydantic import BaseModel, JsonValue
from langchain_openai import ChatOpenAI
from faker import Faker

from typing import Any, Dict
from pydantic import BaseModel
from pydantic import RootModel
from langchain_core.output_parsers import PydanticOutputParser, JsonOutputParser
import pickle
import importlib
from langchain_tavily import TavilySearch
import re
from typing import List, Literal, Optional, Union
from langchain.tools import BaseTool
import faker_commerce

faker = Faker()
faker.add_provider(faker_commerce.Provider)
# print(faker.ecommerce_name()) 



In [3]:
load_dotenv()

True

In [4]:
import os
import sys

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if parent_dir not in sys.path:
    print("Adding parent directory to sys.path")
    sys.path.append(parent_dir)

Adding parent directory to sys.path


In [5]:
import backend.prompts as prompts
importlib.reload(prompts)

PROMPT_REGISTRY = prompts.PROMPT_REGISTRY

In [9]:
def update_prompt_registry():
    importlib.reload(prompts)
    global PROMPT_REGISTRY
    PROMPT_REGISTRY = prompts.PROMPT_REGISTRY

In [10]:
update_prompt_registry()

In [11]:
PROMPT_REGISTRY["ui_generation_v2"]

'\nYou are a highly specialized **UI Generation Agent** designed to create high-fidelity UI specifications. Your primary task is to translate user requirements and provided JSON data into a structured, executable UI definition.\n\n---\n\n## Core Instruction\n\nGenerate a single, comprehensive **JSON object** that represents the complete UI specification.\nThe output MUST:\n- Strictly follow the `UISpec` Pydantic schema.\n- Contain a top-level key `root`.\n- Be a single compact JSON object with no extra whitespace, indentation, or markdown syntax.\n\nLogical grouping is mandatory when multiple UI elements or data points are involved.\n\n---\n\n## Component Library & Usage Rules\n\nYou may only use the components defined below. Select each component based on the `usage` description.\n\n\n### Root Component\nUsage: Top-level container for the entire UI.\n{\n  "type": "root",\n  "children": [ ...components ]\n}\n\n### Container\nUsage: Group related components together.\n{\n  "type": "cont

In [12]:
DATA_REGISTRY = {}

{'description': 'Get a list of products.\n\nReturns:\n    list[Dict[str, Any]]: A list of product dictionaries.',
 'properties': {},
 'title': 'get_products',
 'type': 'object'}

In [ ]:
def clean_tavily_results(results: Dict) -> List[Dict[str, str]]:
    cleaned_results = []
    cleaned_results.append({
        "title": results["query"],
        "content": results["answer"],
    })
    for result in results["results"]:
        cleaned_results.append({
            "title": result["title"],
            "content": re.sub(r'http\S+', '', result["content"]),
        })
    return cleaned_results

@tool
def web_search(query: str) -> List[Dict[str, str]]:
    """Perform a web search and return the results.
    Args:
        query (str): The search query.
    Returns:
        List[Dict[str, str]]: A list of dictionaries containing the search results.
    """
    # print(f"\n[WEB SEARCH] Query: {query}")
    try:
        search = TavilySearch( max_results=5, include_answer=True )
        results = search.invoke({"query": query})
        cleaned_results = clean_tavily_results(results)
        print(f"[WEB SEARCH] Results: {cleaned_results}")
        return cleaned_results
    except Exception as e:
        print(f"Error during web search: {str(e)}")
        return f"Error during web search: {str(e)}"

@tool
def get_products() -> list[Dict[str, Any]]:
    """
    Get a list of products.

    Returns:
        list[Dict[str, Any]]: A list of product dictionaries.
    """
    fake = faker.Faker()
    products = []
    for _ in range(10):
        product = {
            "name": fake.word().title(),
            "price": round(fake.random_number(digits=2), 2),
            "description": fake.sentence(nb_words=10),
            "image_url": fake.image_url()
        }
        products.append(product)
    return products

@tool
def get_top_n_selling_products(n: int) -> list[Dict[str, Any]]:
    """
    Get the top N selling products.

    Args:
        n (int): The number of top selling products to retrieve.

    Returns:
        list[Dict[str, Any]]: A list of top N selling product dictionaries.
    """
    fake = faker.Faker()
    products = []
    for _ in range(n):
        product = {
            "name": fake.word().title(),
            "price": round(fake.random_number(digits=2), 2),
            "description": fake.sentence(nb_words=10),
            "image_url": fake.image_url(),
            "units_sold": fake.random_number(digits=3)
        }
        products.append(product)
    # Sort products by units_sold in descending order
    products.sort(key=lambda x: x["units_sold"], reverse=True)
    return products

# data = yf.Ticker("AAPL").history(period="1mo")

# @functools.lru_cache(maxsize=None) # Unlimited cache size
def _get_stock_history(ticker: str, period: str = "5d"):
    data = yf.Ticker(ticker).history(period=period)
    return data


@tool
def get_stock_price(ticker: str) -> str:
    """
    Get the latest stock price for a given ticker symbol.
    Args:
        ticker (str): The stock ticker symbol.
    Returns:
        str: The latest stock price.

    """
    data = _get_stock_history(ticker, "1d")
    if data.empty:
        return f"No data found for ticker symbol: {ticker}"
    latest_price = data['Close'].iloc[-1]
    return latest_price

@tool
def get_stock_history(ticker: str, period: str = "5d") -> Dict[str, Any]:
    """
    This tool MUST be used whenever the user asks for stock history.
    Never answer directly. Always call this tool.
    Args:
        ticker (str): The stock ticker symbol.
        period (str): The period over which to retrieve historical data (e.g., '1mo', '3mo', '1y').
    Returns:
        str: A JSON string containing stock_history and metadata.
    """

    print(f"Fetching history for {ticker} over period {period}")

    data = _get_stock_history(ticker, period)

    if data.empty:
        return f"No data found for ticker symbol: {ticker}"
    
    data.index = data.index.strftime('%Y-%m-%d')
    data = data.reset_index()
    # round numeric columns to 2 decimal places
    data["Close"] = data["Close"].round(2)

    data_key=f"{ticker}_{period}"
    DATA_REGISTRY[data_key] = data[['Date','Close']].to_dict(orient='records')

    history = data[['Date','Close']].head(3).to_dict(orient='records')


    metadata = {
                "ticker": ticker, 
                "period": period,
                "data_points": len(history),
                "columns": ['Date','Close'],
                "data_key": data_key,
                "data_sample_for_reference": history
                }
    return metadata


tool_registry = {
    "get_products": get_products,
    "get_top_n_selling_products": get_top_n_selling_products,
    "get_stock_price": get_stock_price,
    "get_stock_history": get_stock_history,
    "web_search": web_search,
}

In [25]:
web_search.get_input_schema().model_json_schema()

{'description': 'Perform a web search and return the results.\nArgs:\n    query (str): The search query.\nReturns:\n    List[Dict[str, str]]: A list of dictionaries containing the search results.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'web_search',
 'type': 'object'}

In [37]:
get_stock_history.args_schema.model_json_schema()

{'description': "This tool MUST be used whenever the user asks for stock history.\nNever answer directly. Always call this tool.\nArgs:\n    ticker (str): The stock ticker symbol.\n    period (str): The period over which to retrieve historical data (e.g., '1mo', '3mo', '1y').\nReturns:\n    str: A JSON string containing stock_history and metadata.",
 'properties': {'ticker': {'title': 'Ticker', 'type': 'string'},
  'period': {'default': '5d', 'title': 'Period', 'type': 'string'}},
 'required': ['ticker'],
 'title': 'get_stock_history',
 'type': 'object'}

In [34]:

def extract_tool_schema(tool: BaseTool):
    data=tool.get_input_schema().model_json_schema()
    return data

extract_tool_schema(get_stock_history)

{'description': "This tool MUST be used whenever the user asks for stock history.\nNever answer directly. Always call this tool.\nArgs:\n    ticker (str): The stock ticker symbol.\n    period (str): The period over which to retrieve historical data (e.g., '1mo', '3mo', '1y').\nReturns:\n    str: A JSON string containing stock_history and metadata.",
 'properties': {'ticker': {'title': 'Ticker', 'type': 'string'},
  'period': {'default': '5d', 'title': 'Period', 'type': 'string'}},
 'required': ['ticker'],
 'title': 'get_stock_history',
 'type': 'object'}

In [14]:
llm_google = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    streaming=True
)


llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://aibe.mygreatlearning.com/openai/v1",
    model='gpt-4o-mini',
    temperature=0
)

ui_generation_llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://aibe.mygreatlearning.com/openai/v1",
    model='gpt-4o',
    temperature=0,
           streaming=True
)
# .bind(
#     # May instruct the LLM, but DO NOT send json_schema
#     response_format={"type": "json_object"}  
# )


# ui_generation_llm = ChatGoogleGenerativeAI(
#     model="gemini-2.5-flash",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
# )


In [12]:
llm

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x32bb07290>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x32f5d6240>, root_client=<openai.OpenAI object at 0x1076e19a0>, root_async_client=<openai.AsyncOpenAI object at 0x32bba4920>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [13]:
ui_generation_llm

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x32f789d90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x32f789fd0>, root_client=<openai.OpenAI object at 0x32f789b50>, root_async_client=<openai.AsyncOpenAI object at 0x32f789d60>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True, streaming=True)

In [ ]:
tool_router_prompt = """
You are a Tool Router.

Given a user query, select the relevant TOOL GROUPS.

Do NOT select individual tools.
Select only group names.
"""

llm.invoke(tool_router_prompt)

In [ ]:
from typing import List, Dict, Any, Optional, Literal
from pydantic import BaseModel, Field

class ToolStep(BaseModel):
    step_id: str
    type: Literal["tool", "respond"]
    tool_name: str | None = None
    arguments: dict | None = None
    depends_on: Optional[List[str]] = Field(
        default=None,
        description="Step IDs this step depends on"
    )

class PlanningSchema(BaseModel):
    strategy: Literal["parallel", "sequential", "hybrid"]
    steps: List[ToolStep]

In [ ]:
tool_registry = [
    {
        "name": "get_user_profile",
        "description": "Fetch user profile information",
        "args": {
            "user_id": "string"
        },
        "returns": "UserProfile"
    },
    {
        "name": "get_recent_transactions",
        "description": "Fetch recent user transactions",
        "args": {
            "user_id": "string",
            "limit": "integer"
        },
        "returns": "Transaction[]"
    }
]

In [ ]:
class IndividualTask(BaseModel):
    task_id: str
    description: str
    confidence_score: float

class TaskList(BaseModel):
    task : list[IndividualTask]


PLANNER_SYSTEM_PROMPT = """You are a Planning Agent.

You are given a list of AVAILABLE TOOLS with their names, descriptions,
and input/output schemas.

You can REFER to tools, but you CANNOT execute them.

Your job is to decide:
- Which tools are needed
- In what order
- Which tools can run in parallel
- What arguments each tool needs

Output ONLY a JSON plan that matches the PlanningSchema.
"""

planner_messages = [
    {
        "role": "system",
        "content": PLANNER_SYSTEM_PROMPT
    },
    {
        "role": "system",
        "content": (
            "AVAILABLE TOOLS (read-only, do NOT execute):\n"
            f"{json.dumps(tool_registry, indent=2)}"
        )
    },
    {
        "role": "user",
        "content": user_query
    }
]



In [67]:
llm_planning_agent = llm.with_structured_output(TaskList)

# messages = [
#     {"role": "system", "content": planning_agent_system_message},
#     {"role": "user", "content": "Show me list of all the product available in the store and provide me the details of the top 5 products based on customer ratings."}
# ]

messages = [
    {"role": "system", "content": planning_agent_system_message},
    {"role": "user", "content": "Get me the stock price history for AAPL for last 1 month and also provide me the latest stock price. Do not use charts"}
]

# provide step by step instructions to bake a cake and also get me a list of ingredients required to bake that cake.
# messages = [
#     {"role": "system", "content": planning_agent_system_message},
#     {"role": "user", "content": "Provide step by step instructions to bake a cake and also get me a list of ingredients required to bake that cake."}
# ]

# messages = [
#     {"role": "system", "content": planning_agent_system_message},
#     {"role": "user", "content": "Compare Nvidia GPUs with AMD GPUs in terms of performance and price. Use web search to get the latest information."}
# ]

# messages = [
#     {"role": "system", "content": planning_agent_system_message},
#     {"role": "user", "content": "Compare Nvidia GPU 3090 with AMD Radeon RX 6800 XT in terms of performance and price. Use web search to get the latest information and provide comparison in tabular format."}
# ]


response = llm_planning_agent.invoke(messages)

In [68]:
print(response.model_dump_json(indent=2))

{
  "task": [
    {
      "task_id": "1",
      "description": "Get the stock price history for AAPL for the last 1 month",
      "confidence_score": 0.9
    },
    {
      "task_id": "2",
      "description": "Provide the latest stock price for AAPL",
      "confidence_score": 0.9
    }
  ]
}


In [69]:
tasks = response.model_dump().get("task", [])
for task in tasks:
    print(f"Task ID: {task['task_id']}, Description: {task['description']}, Confidence Score: {task['confidence_score']}")

Task ID: 1, Description: Get the stock price history for AAPL for the last 1 month, Confidence Score: 0.9
Task ID: 2, Description: Provide the latest stock price for AAPL, Confidence Score: 0.9


In [ ]:
tool_calling_agent_system_message = """
You are a Tool Calling Agent. Your goal is to determine if any of the tasks provided can be accomplished using the available tools and call the appropriate tool with the necessary parameters.
If a task can be accomplished without using a tool, provide a direct answer in non markdown format.
"""

tools = [get_products,get_top_n_selling_products,get_stock_price,get_stock_history,web_search]

tool_calling_agent = llm.bind_tools(tools)

In [ ]:
tool_results = []
for task in tasks:
    task_description = task['description']
    print(f"\nProcessing Task ID: {task['task_id']}, Description: {task_description}")
    response = tool_calling_agent.invoke([
        {"role": "system", "content": tool_calling_agent_system_message},
        {"role": "user", "content": task_description}
    ])
    print(f"Response for Task ID {task['task_id']}:\n{response}\n")
    if response.tool_calls:
        for tool_call in response.tool_calls:
            name = tool_call["name"]
            args = tool_call["args"]

            tool = tool_registry.get(name)
            if tool is None:
                print(f"⚠️ Unknown tool requested: {name}")
                continue

            # Invoke dynamically
            result = tool.invoke(args)

            print("Tool result:", result)
            tool_results.append({
                "task_description": task_description,
                "tool_name": name,
                "args": args,
                "tool_result": result
            })

    elif response.content:
        print(f"No tool call made for Task ID {task['task_id']}. Response: {response.content}")
        tool_results.append({
            "task_description": task_description,
            "response": response.content
        })
    


Processing Task ID: 1, Description: Get the stock price history for AAPL for the last 1 month
Response for Task ID 1:
content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 429, 'total_tokens': 450, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'id': 'chatcmpl-ClaugizTcDXRBnwWFtdTMkNWEIaMV', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--50a72a47-73f3-4181-9029-90d12d3a5d1d-0' tool_calls=[{'name': 'get_stock_history', 'args': {'ticker': 'AAPL', 'period': '1mo'}, 'id': 'call_hM3j2M120ymnaLai5kicGt5T', 'type': 'tool_call'}] usage_metadata={'input_tokens': 429, 'output_tokens': 21, 'total_tokens': 450, 'input_token_de

In [72]:
tool_results

[{'task_description': 'Get the stock price history for AAPL for the last 1 month',
  'tool_name': 'get_stock_history',
  'args': {'ticker': 'AAPL', 'period': '1mo'},
  'tool_result': {'ticker': 'AAPL',
   'period': '1mo',
   'data_points': 3,
   'columns': ['Date', 'Close'],
   'data_key': 'AAPL_1mo',
   'data_sample_for_reference': [{'Date': '2025-11-10', 'Close': nan},
    {'Date': '2025-11-11', 'Close': 275.25},
    {'Date': '2025-11-12', 'Close': 273.47}]}},
 {'task_description': 'Provide the latest stock price for AAPL',
  'tool_name': 'get_stock_price',
  'args': {'ticker': 'AAPL'},
  'tool_result': 278.7799987792969}]

In [73]:
print(json.dumps(tool_results, indent=2))

[
  {
    "task_description": "Get the stock price history for AAPL for the last 1 month",
    "tool_name": "get_stock_history",
    "args": {
      "ticker": "AAPL",
      "period": "1mo"
    },
    "tool_result": {
      "ticker": "AAPL",
      "period": "1mo",
      "data_points": 3,
      "columns": [
        "Date",
        "Close"
      ],
      "data_key": "AAPL_1mo",
      "data_sample_for_reference": [
        {
          "Date": "2025-11-10",
          "Close": NaN
        },
        {
          "Date": "2025-11-11",
          "Close": 275.25
        },
        {
          "Date": "2025-11-12",
          "Close": 273.47
        }
      ]
    }
  },
  {
    "task_description": "Provide the latest stock price for AAPL",
    "tool_name": "get_stock_price",
    "args": {
      "ticker": "AAPL"
    },
    "tool_result": 278.7799987792969
  }
]


In [74]:
from pydantic import BaseModel, Field, model_validator
from typing import Annotated
from pydantic import Field

# --- Base and Auxiliary Models ---

class BaseComponent(BaseModel):
    # This must be defined as str first, and then overridden by Literals in subclasses
    type: str
    # usage: Optional[str] = None

    model_config = {
        "extra": "forbid"
    }

# class TableCell(BaseModel):
#     key: str
#     # Note: Using `str | int | float | bool | None` is the modern Pydantic/Python way
#     value: str | int | float | bool | None

# class TableRow(BaseModel):
#     cells: List[TableCell]


# Helper classes for inline data structure
class TableColumn(BaseModel):
    header: str
    key: str # A key to reference the value in the row list

class TableRow(BaseModel):
    # This will be a dictionary where keys match TableColumn.key
    # e.g., {"product_name": "Laptop", "price": 1200}
    pass # Use dict[str, Any] in actual implementation if possible

# --- Component Definitions (Uses string literal "Component" for recursion) ---

class Container(BaseComponent):
    type: Literal["container"]
    variant: Literal["vertical", "horizontal"]
    children: List["Component"]


class Text(BaseComponent):
    type: Literal["text"]
    variant: Literal[
        "header",
        "subheader",
        "paragraph",
        "span",
        "span-bold",
        "span-italic",
        "span-underline"
    ]

    value: Optional[str] = None
    children: Optional[List["Text"]] = None

    model_config = {
        "extra": "forbid"
    }

    @model_validator(mode="after")
    def validate_value_or_children(self):
        if self.value is not None and self.children is not None:
            raise ValueError("Text component cannot have both 'value' and 'children'")
        if self.value is None and self.children is None:
            raise ValueError("Text component must have either 'value' or 'children'")
        return self



class Chart(BaseComponent):
    type: Literal["chart"]
    variant: Literal["line", "bar"]
    x_axis: str
    y_axis: str
    title: str
    dataKey: str

# class SimpleTable(BaseComponent):
#     type: Literal["simple-table"]
#     title: str
#     columns: List[str]
#     rows: List[TableRow]

# class DataTable(BaseComponent):
#     type: Literal["data-table"]
#     title: str
#     dataKey: str

# class Table(BaseComponent):
#     type: Literal["table"]
#     title: str
#     dataKey: str

class TableColumn(BaseModel):
    header: str
    key: str # A key to reference the value in the row list

class TableRow(BaseModel):
    pass 

class EmbedTable(BaseComponent):
    type: Literal["embed-table"]
    title: str
    columns: List[TableColumn]
    rows: List[Dict[str, Any]]

class DataTable(BaseComponent):
    type: Literal["data-table"]
    title: str
    dataKey: str



class Card(BaseComponent):
    type: Literal["card"]
    title: str
    content: str


class Button(BaseComponent):
    type: Literal["button"]
    label: str
    action: str


class Image(BaseComponent):
    type: Literal["image"]
    src: str
    alt: str


class ListComponent(BaseComponent):
    type: Literal["list"]
    children: List["Component"]

class Root(BaseComponent):
    type: Literal["root"]
    children: List["Component"]

# --- Component Union Definition ---

# Define the Union using the model classes
Component = Annotated[
    Union[
        Root,
        Container,
        Text,
        Chart,
        EmbedTable,
        DataTable,
        Card,
        Button,
        Image,
        ListComponent,
    ],
    Field(discriminator="type")
]

# --- Resolve Forward References ---

# The rebuild calls are ESSENTIAL to link the string "Component" 
# to the actual Component Union type.
Root.model_rebuild()
Container.model_rebuild()
Text.model_rebuild()
ListComponent.model_rebuild()

# --- Final Output Schema ---

class UISpec(BaseModel):
    root: Root

    model_config = {
        "extra": "forbid"
    }


basic_ui_spec_v3 = """
### Root Component
{
  "type": "root",
  "usage": "Top-level container for the entire UI.",
  "children": [ ...components ]
}

### Container
{
  "type": "container",
  "usage": "Group related components together.",
  "variant": "vertical" | "horizontal",
  "children": [ ...components ]
}

### Text Component (Supports Nested Inline Formatting)
{
  "type": "text",
  "variant": "header" | "subheader" | "paragraph" | "span" | "span-bold" | "span-italic" | "span-underline",
  "usage": "Display textual content. Use 'span' variants for inline formatting inside other text.",
  "value": "string",
  "children": [ ...textComponents ]
}

Rules for `text`:
1. A Text component must have **either**:
   - `value` (plain text), OR
   - `children` (nested text components)
2. Never include both `value` and `children` in the same Text component.
3. Use `children` when inline formatting (bold, italic, underline) is needed.
4. Variants:
   - Use `header` / `subheader` / `paragraph` for block text.
   - Use `span` for normal inline text.
   - Use `span-bold`, `span-italic`, `span-underline` for styled inline text.

### Chart
{
  "type": "chart",
  "usage": "Visualize data. Choose correct x_axis and y_axis from provided data.",
  "variant": "line" | "bar",
  "x_axis": "string",
  "y_axis": "string",
  "title": "string",
  "dataKey": "chart_data_key"
}

### Data Table
{
  "type": "data-table",
  "usage": "Display tabular data. Use 'dataKey' for large datasets",
  "title": "string",
  "dataKey": "table_data_key", //A key referencing the dataset returned by a tool
}

### Embed Table
{
  "type": "embed-table",
  "usage": "Display small/static tabular data.",
  "title": "string",
  "columns": [
    {
      "header": "string",
      "key": "string"
    }, ...
  ],
  "rows": [
    { "column_key": "value", ... }, ...
  ]
}


### Card
{
  "type": "card",
  "usage": "Present concise information.",
  "title": "string",
  "content": "string"
}

### Button
{
  "type": "button",
  "usage": "Trigger an action or event.",
  "label": "string",
  "action": "string"
}

### Image
{
  "type": "image",
  "usage": "Display visual content.",
  "src": "image_url",
  "alt": "string"
}

### List
{
  "type": "list",
  "usage": "Display a collection of items.",
  "children": [ ...components ]
}
"""

ui_generation_system_message_v2 = f"""
You are a highly specialized **UI Generation Agent** designed to create high-fidelity UI specifications. Your primary task is to translate user requirements and provided JSON data into a structured, executable UI definition.

---

## Core Instruction
Generate a single, comprehensive **JSON object** that represents the complete UI specification.
The output MUST:
- Strictly follow the `UISpec` Pydantic schema.
- Contain a top-level key `root`.
- Be a single compact JSON object with no extra whitespace, indentation, or markdown syntax.

Logical grouping is mandatory when multiple UI elements or data points are involved.

---

## Component Library & Usage Rules

You may only use the components defined below. Select each component based on the `usage` description.

{basic_ui_spec_v3}

---

## Output Principles

### 1. Structure
- Output MUST be a compact single JSON object matching `UISpec`.
- No extra fields beyond schema.
- `root` is always the top-level key.

### 2. Data Integration
- Charts: Must use `dataKey` for datasets. Only use dataKey if available in the dataset
- Use `data-table` ONLY when the dataset is referenced using a `dataKey`.
- Use `embed-table` ONLY when the tabular data is provided inline as arrays (`columns` and `rows`).
- NEVER mix fields between these two table types.
- A `data-table` MUST NOT contain `columns` or `rows`.
- An `embed-table` MUST NOT contain `dataKey`.

### 3. Text Nesting
- Use nested `text.children` when inline styling is needed.
- Inline formatted text MUST use:
  - `span-bold`
  - `span-italic`
  - `span-underline`

### 4. Hierarchy
- Components must be organized logically using `root` and `container`.

### 5. Actionability
- Every `button` must clearly indicate its purpose through its `label`.
"""

In [84]:
update_prompt_registry()
ui_generation_system_message = PROMPT_REGISTRY["ui_generation_v2"]

structured_ui_gen_llm = ui_generation_llm.with_structured_output(UISpec, method="function_calling")

messages = [
    {"role": "system", "content": ui_generation_system_message_v2},
    {"role": "user", "content": f"Generate a UI specification for the following data:\n  {json.dumps(tool_results, ensure_ascii=False)}. Use data-table for showing stock history "}
]
# res = structured_ui_gen_llm.invoke(messages)



In [85]:
print(ui_generation_system_message)


You are a highly specialized **GUI Generation Agent** designed to create high-fidelity UI specifications. Your primary task is to translate user requirements and provided JSON data into a structured, executable UI definition.

---

## Core Instruction
Generate a single, comprehensive **JSON object** that represents the complete UI specification.
The output MUST:
- Strictly follow the `UISpec` Pydantic schema.
- Contain a top-level key `root`.
- Be a single compact JSON object with no extra whitespace, indentation, or markdown syntax.

Logical grouping is mandatory when multiple UI elements or data points are involved.

---

## Component Library & Usage Rules

You may only use the components defined below. Select each component based on the `usage` description.


### Root Component
{
  "type": "root",
  "usage": "Top-level container for the entire UI.",
  "children": [ ...components ]
}

### Container
{
  "type": "container",
  "usage": "Group related components together.",
  "variant"

In [77]:
json.dumps(tool_results)

'[{"task_description": "Get the stock price history for AAPL for the last 1 month", "tool_name": "get_stock_history", "args": {"ticker": "AAPL", "period": "1mo"}, "tool_result": {"ticker": "AAPL", "period": "1mo", "data_points": 3, "columns": ["Date", "Close"], "data_key": "AAPL_1mo", "data_sample_for_reference": [{"Date": "2025-11-10", "Close": NaN}, {"Date": "2025-11-11", "Close": 275.25}, {"Date": "2025-11-12", "Close": 273.47}]}}, {"task_description": "Provide the latest stock price for AAPL", "tool_name": "get_stock_price", "args": {"ticker": "AAPL"}, "tool_result": 278.7799987792969}]'

In [78]:
messages

[{'role': 'system',
  'content': '\nYou are a highly specialized **UI Generation Agent** designed to create high-fidelity UI specifications. Your primary task is to translate user requirements and provided JSON data into a structured, executable UI definition.\n\n---\n\n## Core Instruction\nGenerate a single, comprehensive **JSON object** that represents the complete UI specification.\nThe output MUST:\n- Strictly follow the `UISpec` Pydantic schema.\n- Contain a top-level key `root`.\n- Be a single compact JSON object with no extra whitespace, indentation, or markdown syntax.\n\nLogical grouping is mandatory when multiple UI elements or data points are involved.\n\n---\n\n## Component Library & Usage Rules\n\nYou may only use the components defined below. Select each component based on the `usage` description.\n\n\n### Root Component\n{\n  "type": "root",\n  "usage": "Top-level container for the entire UI.",\n  "children": [ ...components ]\n}\n\n### Container\n{\n  "type": "container

In [60]:
res = structured_ui_gen_llm.invoke(messages)

In [79]:
chunks = []
for chunk in structured_ui_gen_llm.stream(messages):
        chunks.append(chunk)
        # print(chunk.model_dump(), end="", flush=True)

In [80]:
print(res.model_dump_json(indent=2))

{
  "root": {
    "type": "root",
    "children": [
      {
        "type": "container",
        "variant": "vertical",
        "children": [
          {
            "type": "text",
            "variant": "header",
            "value": "GPU Comparison: Nvidia RTX 3090 vs AMD Radeon RX 6800 XT",
            "children": null
          },
          {
            "type": "container",
            "variant": "horizontal",
            "children": [
              {
                "type": "card",
                "title": "Nvidia RTX 3090",
                "content": "The GeForce RTX 3090 offers high performance but is expensive, with a typical price around $1,500. It's often considered less value for money compared to other GPUs."
              },
              {
                "type": "card",
                "title": "AMD Radeon RX 6800 XT",
                "content": "The AMD Radeon RX 6800 XT has a 3DMark score of 3675, ranks 40, and costs $649. It competes with the RTX 4070 in performance

In [81]:
print(chunks[-1].model_dump_json(indent=2))

{
  "root": {
    "type": "root",
    "children": [
      {
        "type": "container",
        "variant": "vertical",
        "children": [
          {
            "type": "text",
            "variant": "header",
            "value": "AAPL Stock Information",
            "children": null
          },
          {
            "type": "container",
            "variant": "horizontal",
            "children": [
              {
                "type": "card",
                "title": "Latest Stock Price",
                "content": "$278.78"
              },
              {
                "type": "data-table",
                "title": "AAPL Stock Price History (Last 1 Month)",
                "dataKey": "AAPL_1mo"
              }
            ]
          }
        ]
      }
    ]
  }
}


In [82]:
chunks_json = [chunk.model_dump() for chunk in chunks]

In [83]:
with open("ui_spec_stream_stock_table.json", "w") as f:
    json.dump(chunks_json, f, indent=2)

In [114]:
print(ui_generation_system_message)


You are a highly specialized **UI Generation Agent** designed to create high-fidelity UI specifications. Your primary task is to translate user requirements and provided JSON data into a structured, executable UI definition.

---

## Core Instruction
Generate a single, comprehensive **JSON object** that represents the complete UI specification.
The output MUST:
- Strictly follow the `UISpec` Pydantic schema.
- Contain a top-level key `root`.
- Be a single compact JSON object with no extra whitespace, indentation, or markdown syntax.

Logical grouping is mandatory when multiple UI elements or data points are involved.

---

## Component Library & Usage Rules

You may only use the components defined below. Select each component based on the `usage` description.


### Root Component
{
  "type": "root",
  "usage": "Top-level container for the entire UI.",
  "children": [ ...components ]
}

### Container
{
  "type": "container",
  "usage": "Group related components together.",
  "variant":